# [0] [Total3D pickles] layout_emitter + 3D lighting

In [1]:
# borrowed from Total3D repo -> notebooks/parse_OR_labels-lighting-reindexed%2Bassigned-V4.ipynb
import sys
sys.path.insert(0, '/home/ruizhu/Documents/Projects/semanticInverse/train')

import os
import json

%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import matplotlib.pyplot as plt
%matplotlib widget

# dataset = 'sunrgbd'
dataset = 'OR'

mode = 'val'
# mode = 'train'
# mode = 'train-tmp'

# split_path_dict = {'sunrgbd': 'data/sunrgbd/preprocessed', 'OR': 'utils_OR/openrooms/preprocessed-full'}
split_path_dict = {'sunrgbd': 'data/sunrgbd/preprocessed', 'OR': '/home/ruizhu/Documents/Projects/semanticInverse/train/data/openrooms/list_OR_V4full/list'}

if dataset == 'sunrgbd':
    split_file = os.path.join(split_path_dict[dataset], mode + '.json')
    with open(split_file) as file:
        split = json.load(file)
elif dataset == 'OR':
    split_file = os.path.join(split_path_dict[dataset], mode + '.txt')
    with open(split_file) as f:
        mylist = f.read().splitlines() 
    split = [x.split(' ') for x in mylist]

print(len(split), split[0])

5664 ['scene0509_00', '1', 'mainDiffLight_xml1/scene0509_00/im_1.hdr', 'main_xml1/scene0509_00/imsemLabel_1.npy']


In [2]:
import torch
import numpy as np
from pathlib import Path
from utils.utils_total3D.utils_OR_visualize import format_layout, Box
from utils.utils_total3D.data_config import OR4XCLASSES_dict, NYU40CLASSES
from utils.utils_total3D.utils_others import OR4X_mapping_catInt_to_RGB, OR4X_mapping_catStr_to_RGB
from utils.utils_total3D.utils_OR_vis_labels import read_vis_scene_3d, RGB_to_01
import pickle

OR = 'OR45'
classes = OR4XCLASSES_dict[OR] if dataset == 'OR' else NYU40CLASSES
with open(str(Path('../train/data/openrooms/total3D_colors/') / OR4X_mapping_catInt_to_RGB['light']), 'rb') as f:
    OR_mapping_catInt_to_RGB = pickle.load(f)[OR]
with open(str(Path('../train/data/openrooms/total3D_colors/') / OR4X_mapping_catStr_to_RGB['light']), 'rb') as f:
    OR_mapping_catStr_to_RGB = pickle.load(f)[OR]

import random
# image_id = random.randint(0, len(split))
# image_id = 3
# pickle_path_ori = split[image_id]

# pickle_id = random.randint(0, len(split))
# pickle_id = 10
# pickle_id = 26
# pickle_id = 27

pickle_id = 16 # blue room, sofa, window
wall_idx, cell_i, cell_j = 2, 3, 3

# pickle_id = 6
# wall_idx, cell_i, cell_j = 4, 2, 4

# pickle_id = 18 # classroom
# wall_idx, cell_i, cell_j = 4, 3, 3

# random
# pickle_id = 25


naming = split[pickle_id][2]

# naming = 'main_xml1/scene0552_00/im_1.hdr'
# naming = 'main_xml1/scene0552_00/im_1.hdr'
# naming = 'mainDiffLight_xml1/scene0017_02/im_4.hdr'
# naming = 'mainDiffMat_xml1/scene0695_03/im_26.hdr'
# pickle_path_ori = '/data/ruizhu/OR-V3-OR45_total3D_train_test_data/%06d.pkl'%pickle_id
meta_split = naming.split('/')[0]
scene_name = naming.split('/')[1]
withinsequence_id = int(naming.split('/')[2].split('.')[0].split('_')[1])
# assert pickle_id==withinsequence_id

train_test_data_path = '/data/ruizhu/OR-V4full-detachEmitterRERERE20210425-OR45_total3D_train_test_data'

pickle_path_ori = '%s/%s/%s/%s'%(train_test_data_path, naming.split('/')[0], naming.split('/')[1], naming.split('/')[2].replace('im', 'layout_obj').replace('hdr', 'pkl'))


grid_size = 8

pickle_path = pickle_path_ori.replace('.pkl', '_reindexed.pkl')
pickle_emitters_path = pickle_path_ori.replace('.pkl', '_emitters.pkl')
pickle_emitters_assign_info_list_path = pickle_path_ori.replace('.pkl', '_emitters_assign_info_%dX%d_V4.pkl'%(grid_size, grid_size))

print(pickle_path)

# ======= read saved results from evaluation
# results_folder = '20210427-142731--EVAL-20210427-011227'; tid=34000
# results_folder = '20210427-041449-train_POD_emitterEst_lightAccuV3_bs2x4_GTBRDF-light-Light_FIXFIXFIXED2axis_overfitValMini_RE2'; tid=12000

# EST
# results_folder = '20210427-025334-train_POD_emitterEst_lightAccuV3_bs2x4_RuitrainedBRDF-light-Light_FIXFIXFIXED2axis'; tid=44000
# GT
# results_folder = '20210427-025239-train_POD_emitterEst_lightAccuV3_bs4x4_GT-BRDF-Light_FIXFIXFIXED2axis'; tid=24000

# GT envmap input: 20210428-170647-train_POD_emitterEst_lightAccuV3_bs2x4_GTsampledEnvMap_FIXFIXFIXEDaxis
results_folder = '20210429-214740--EVAL-20210428-170647'; tid=0

layout_results_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_layout_info_tid%d-%d.pickle'%(results_folder, tid, pickle_id)
emitter_results_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_cells_vis_info_list_tid%d-%d.pickle'%(results_folder, tid, pickle_id)

with open(layout_results_pickle_path, 'rb') as f:
    layout_pickle_dict = pickle.load(f)
    print(layout_pickle_dict.keys())
with open(emitter_results_pickle_path, 'rb') as f:
    emitter_pickle_dict = pickle.load(f)
    print(emitter_pickle_dict.keys())

cells_vis_info_list = emitter_pickle_dict['cells_vis_info_list']
cam_R_pred = layout_pickle_dict['pre_cam_R']
pred_layout = layout_pickle_dict['pre_layout']
emitter_cls_prob_PRED = emitter_pickle_dict['emitter_cls_prob_PRED']
cell_info_grid_PRED = emitter_pickle_dict['cell_info_grid_PRED']

results_emitter_input_pickle_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_emitter_input_%d.pickle'%(results_folder, pickle_id)

with open(results_emitter_input_pickle_path, 'rb') as f:
    results_emitter_input_pickle_dict = pickle.load(f)
    
env_scale, hdr_scale = results_emitter_input_pickle_dict['env_scale'], results_emitter_input_pickle_dict['hdr_scale']
print(env_scale, hdr_scale)

/data/ruizhu/OR-V4full-detachEmitterRERERE20210425-OR45_total3D_train_test_data/mainDiffLight_xml/scene0385_01/layout_obj_32_reindexed.pkl
dict_keys(['rgb_img_path', 'bins_tensor', 'est_data', 'gt_cam_R', 'cam_K', 'gt_layout', 'pre_layout', 'pre_cam_R', 'image'])
dict_keys(['cells_vis_info_list', 'cell_info_grid_GT', 'cell_info_grid_PRED', 'emitter_cls_prob_PRED', 'emitter_cls_prob_GT', 'pre_layout', 'pre_cam_R', 'envmap_lightAccu_mean_vis_GT'])
119.1801 0.1263157894736842


In [3]:
import pickle
with open(pickle_path, 'rb') as f:
    sequence = pickle.load(f)
print(sequence.keys())

with open(pickle_emitters_path, 'rb') as f:
    sequence_emitters = pickle.load(f)
print(sequence_emitters.keys(), sequence_emitters['sequence_name'], sequence_emitters['withinsequence_id'])
with open(pickle_emitters_assign_info_list_path, 'rb') as f:
    sequence_emitter2wall_assign_info_dict = pickle.load(f)
print(sequence_emitter2wall_assign_info_dict.keys())

import pickle5 as pickle
emitter_representation_type = '0ambient'
emitters_prop_dict_representation_dict_path = pickle_path.replace('layout_obj', ('emitters_prop_dict_%s'%emitter_representation_type)).replace('_reindexed', '')
with open(emitters_prop_dict_representation_dict_path, 'rb') as f:
    emitters_prop_dict_representation_dict = pickle.load(f)

# transform_to_total3d_coords_dict = {'transform_R': frame.transform_R, 'transform_t': frame.transform_t}
transform_to_total3d_coords_dict_path = Path(train_test_data_path) / meta_split / scene_name / ('transform_to_total3d_coords_dict_%d.pkl'%withinsequence_id)
import pickle5 as pickle
with open(transform_to_total3d_coords_dict_path, 'rb') as f:
    transform_to_total3d_coords_dict = pickle.load(f)
    
# RAW_data_path = '/newfoundland2/ruizhu/siggraphasia20dataset/layout_labels_V4full-detachEmitterRERERE20210420/'
# emitter_dict_pickle_file = Path(RAW_data_path) / meta_split / scene_name / ('emitter_dict_%s_%d.pickle'%('0ambient', withinsequence_id))
# with open(emitter_dict_pickle_file, "rb") as f:
#     emitter_dict_RAW = pickle.load(f)


%matplotlib widget

gt_boxes_list = [format_layout(x) for x in sequence['boxes']['bdb3D']]
for x, class_id in zip(gt_boxes_list, sequence['boxes']['size_cls']):
    x.update({'class_id': class_id})
gt_boxes_dict = {}
for key in list(gt_boxes_list[0].keys()):
    gt_boxes_dict[key] = [x[key] for x in gt_boxes_list]
    
gt_layout = sequence['layout']['bdb3D']

from utils.utils_total3D.data_config import Dataset_Config
from utils.utils_total3D.utils_OR_layout import to_dict_tensor

from utils.utils_total3D.utils_OR_cam import get_rotation_matrix_gt

OR = 'OR45'
dataset_config = Dataset_Config(dataset, OR=OR, version='V3', paths={'total3D_lists_path': '../train/data/openrooms/list_OR_V4full', 'layout_emitter_path': '/data/ruizhu/OR-V4full-OR45_total3D_train_test_data'})
bins_tensor = to_dict_tensor(dataset_config.bins, if_cuda=False)
cam_R_gt = get_rotation_matrix_gt(bins_tensor,
                                  torch.zeros([1]).long()+sequence['camera']['pitch_cls'], torch.zeros([1])+sequence['camera']['pitch_reg'],
                                  torch.zeros([1]).long()+sequence['camera']['roll_cls'], torch.zeros([1])+sequence['camera']['roll_reg']).squeeze().numpy()

# ======== emitters_list
emitters_obj_list = []
for x in range(sequence_emitters['boxes']['bdb3D'].shape[0]):
    obj_dict_new = {'obj_box_3d': sequence_emitters['boxes']['bdb3D_emitter_part'][x], \
                    'random_id': sequence_emitters['boxes']['random_id'][x], \
                    'emitter_prop': sequence_emitters['boxes']['emitter_prop'][x], \
                    'bdb3D_emitter_part': sequence_emitters['boxes']['bdb3D_emitter_part'][x], \
                    'cat_id': sequence_emitters['boxes']['size_cls'][x], \
                    'cat_name': classes[sequence_emitters['boxes']['size_cls'][x]], 'cat_color': RGB_to_01(OR_mapping_catInt_to_RGB[sequence_emitters['boxes']['size_cls'][x]])}
    obj_random_id = sequence_emitters['boxes']['random_id'][x]
    emitter_prop_total3d = emitters_prop_dict_representation_dict[obj_random_id]['emitter_prop_total3d']
    if sequence_emitters['boxes']['emitter_prop'][x]['obj_type'] == 'window':
        light_center_world_total3d = emitter_prop_total3d['light_center_world_total3d'].reshape(3, 1)
        light_axis_world_total3d = emitter_prop_total3d['light_axis_world_total3d'].reshape(3, 1)
        print(light_axis_world_total3d)
    else:
        light_center_world_total3d = np.zeros((3, 1), dtype=np.float32)
        light_axis_world_total3d = np.zeros((3, 1), dtype=np.float32)
    obj_dict_new['light_world_total3d_centeraxis'] = [light_center_world_total3d, light_axis_world_total3d]
    
    obj_dict_new['emitter_prop'].update({'emitter_rgb_float': emitter_prop_total3d['intensity']})

    emitters_obj_list.append(obj_dict_new)
    
emitter2wall_assign_info_list = sequence_emitter2wall_assign_info_dict['emitter2wall_assign_info_list']
cell_info_grid_GT_includeempty = sequence_emitter2wall_assign_info_dict['cell_info_grid']
cell_info_grid_GT = []
for wall_idxx in range(6):
    for i in range(grid_size):
        for j in range(grid_size):
            cell_info = cell_info_grid_GT_includeempty[wall_idxx * grid_size**2 + i * grid_size + j]
            if cell_info['obj_type'] is None:
                continue
            cell_info['wallidx_i_j'] = (wall_idxx, i, j)
            cell_info_grid_GT.append(cell_info)
                
            obj_random_id = cell_info['emitter_info']['random_id']
            emitter_prop_total3d = emitters_prop_dict_representation_dict[obj_random_id]['emitter_prop_total3d']


#             cell_intensity = np.array([emitter_prop_total3d['intensity_scale255'] * x * 255.for x in emitter_prop_total3d['intensity_scaled01']]) # intensity_scaled: [0., 1.]
            cell_intensity = np.array(emitter_prop_total3d['intensity']) * env_scale * hdr_scale
            cell_info['emitter_info']['intensity_scalelog'] = np.log(np.clip(np.linalg.norm(cell_intensity.flatten()) + 1., 1., np.inf))
            cell_info['emitter_info']['intensity'] = [x * env_scale * hdr_scale for x in emitter_prop_total3d['intensity']]
            intensity_scale255 = max(cell_info['emitter_info']['intensity']) / 255.
            intensity_scaled01 = [np.clip(x / (intensity_scale255+1e-5) / 255., 0., 1.) for x in cell_info['emitter_info']['intensity']]
            cell_info['emitter_info']['intensity_scaled01'] = intensity_scaled01

            # other representation-specific params
            if cell_info['obj_type'] == 'window':
                cell_info['emitter_info']['lamb'] = emitter_prop_total3d['lamb']
                cell_info['emitter_info']['light_dir_abs'] = emitter_prop_total3d['light_axis_world_total3d'].reshape(3,)
                if emitter_representation_type in ['1ambient']:
                    cell_info['emitter_info']['ambient'] = emitter_prop_total3d['ambient']
                if emitter_representation_type in ['2ambient']:
                    cell_info['emitter_info']['ambientL'] = emitter_prop_total3d['ambientL']
                    cell_info['emitter_info']['ambientR'] = emitter_prop_total3d['ambientR']



plt.figure(figsize=(15, 8))
ax_2d = plt.gca()

grid_prob = sequence_emitter2wall_assign_info_dict['wall_grid_prob'].reshape(6, -1)
import imageio
rgb_img = imageio.imread(sequence['rgb_img_path'])

scene_box = Box(rgb_img, None, sequence['camera']['K'], cam_R_gt, cam_R_pred, gt_layout, gt_layout, gt_boxes_dict, None, 'GT', None, dataset=dataset, OR='OR45', \
                emitters_obj_list=emitters_obj_list, emitter2wall_assign_info_list=emitter2wall_assign_info_list, \
                emitter_cls_prob_PRED=emitter_cls_prob_PRED, emitter_cls_prob_GT=grid_prob, cell_info_grid_GT=cell_info_grid_GT, cell_info_grid_PRED=cell_info_grid_PRED, \
                grid_size=grid_size, \
                paths={'color_file': '../train/data/openrooms/total3D_colors/OR4X_mapping_catInt_to_RGB_light.pkl'})
if_use_plt=True
img_map, _ = scene_box.draw_projected_layout('GT', return_plt=True, if_save=False, save_path='', if_use_plt=if_use_plt, fig_or_ax=ax_2d)
img_map2 = scene_box.draw_projected_bdb3d('GT', return_plt=True, if_save=False, save_path='', if_use_plt=if_use_plt, fig_or_ax=ax_2d)
# plt.show()

fig_3d, ax_3ds_scene = scene_box.draw_3D_scene_plt('both', if_show_objs=False, hide_random_id=False, if_print_log=True)
# ax_3d = ax_3ds[0]
# ax_3ds_scene[0].view_init(elev=-46, azim=86)

# %matplotlib widget
# sys.path.insert(0, '/home/ruizhu/Documents/Projects/semanticInverse/train/SimpleLayout')
# depth_combined, mask_conflict = scene_box.draw_projected_depth('GT', return_plt=True, if_save=False, save_path='', if_vis=True)

wall_cell_idxes = []
for x in cell_info_grid_GT:
    if x['obj_type'] == 'window':
        wall_cell_idxes.append(x['wallidx_i_j'])
print(wall_cell_idxes)

dict_keys(['rgb_img_path', 'depth_map', 'boxes', 'camera', 'layout', 'scene_name', 'withinsequence_id', 'meta_split', 'meta_name', 'sub_name', 'scene_pickle_file', 'cam_pickle_file', 'frame_pickle_file', 'reindex_info_dict'])
dict_keys(['sequence_name', 'withinsequence_id', 'boxes', 'emitter_random_ids']) scene0385_01 32
dict_keys(['emitter2wall_assign_info_list', 'emitters_obj_list', 'wall_grid_prob', 'cell_prob_mean', 'cell_prob', 'cell_count', 'cell_info_grid'])
[[ 0.2981]
 [ 0.9515]
 [-0.076 ]]
[[ 0.316 ]
 [ 0.9101]
 [-0.2683]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[draw_projected_layout] Returned.
 Detected for 32:lamp not in valid_class_ids of dataset: OR
 Detected for 32:lamp not in valid_class_ids of dataset: OR
 Detected for 31:window not in valid_class_ids of dataset: OR
[draw_projected_bdb3d] Returned.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2 1 2
2 1 3
2 1 4
2 1 5
2 2 2
2 2 3
2 2 4
2 2 5
2 3 2
2 3 3
2 3 4
2 3 5
2 4 2
2 4 3
2 4 4
2 4 5
2 5 2
2 5 3
2 5 4
2 5 5
5 2 2
5 2 3
5 2 4
5 2 5
5 3 2
5 3 3
5 3 4
5 3 5
5 4 2
5 4 3
5 4 4
5 4 5
5 5 2
5 5 3
5 5 4
5 5 5
[(2, 1, 2), (2, 1, 3), (2, 1, 4), (2, 1, 5), (2, 2, 2), (2, 2, 3), (2, 2, 4), (2, 2, 5), (2, 3, 2), (2, 3, 3), (2, 3, 4), (2, 3, 5), (2, 4, 2), (2, 4, 3), (2, 4, 4), (2, 4, 5), (2, 5, 2), (2, 5, 3), (2, 5, 4), (2, 5, 5), (5, 2, 2), (5, 2, 3), (5, 2, 4), (5, 2, 5), (5, 3, 2), (5, 3, 3), (5, 3, 4), (5, 3, 5), (5, 4, 2), (5, 4, 3), (5, 4, 4), (5, 4, 5), (5, 5, 2), (5, 5, 3), (5, 5, 4), (5, 5, 5)]


In [ ]:
wall_cell_idxes = []
for x in cell_info_grid_GT:
    if x['obj_type'] == 'window':
        print(x.keys())
        wall_cell_idxes.append(x['wallidx_i_j'])
print(wall_cell_idxes)

In [221]:
basis_v_indexes = [(3, 2, 0), (7, 6, 4), (4, 5, 0), (6, 2, 5), (7, 6, 3), (7, 3, 4)]


# [0] envmap
adapted from Total3D: **/notebooks/parse_OR_labels-lighting-reindexed%2Bassigned%2BenvmapReRender-V4_window-ImproveSG.ipynb**

In [151]:
import sys
sys.path.insert(0, '/home/ruizhu/Documents/Projects/Total3DUnderstanding')

from utils_OR.DatasetCreation import SGOptim
import torch
from pathlib import Path

RAW_path = '/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/DatasetCreation'
xml_path = '/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes'
root_uv_mapped = Path('/newfoundland2/ruizhu/siggraphasia20dataset/uv_mapped')
envmap_root = '/home/ruizhu/Documents/data/EnvDataset/'

cam_dict = sequence['camera']['cam_dict']
cam_rotMat = cam_dict['cam_rotMat'] # world coordinates -> cam coordinates
transform_R, transform_t = transform_to_total3d_coords_dict['transform_R'], transform_to_total3d_coords_dict['transform_t']

emitters_windows = [x for x in emitters_obj_list if x['emitter_prop']['obj_type'] == 'window' and x['emitter_prop']['if_lit_up']]
window_random_ids = [x['random_id'] for x in emitters_windows]
print(window_random_ids)

random_id_chosen_idx = 1 # [!!!!!!!!!!!!!!!!] change this!
random_id_chosen = window_random_ids[random_id_chosen_idx]

emitter_window = emitters_windows[random_id_chosen_idx]

# ==== read light dats
light_path = emitters_prop_dict_representation_dict[random_id_chosen]['emitter_prop']['light_path']
# with open(light_path, 'rb') as f:
#     light_dat = pickle.load(f)
# print(light_path)
# print(light_dat.keys(), light_dat['lamb'])

light_axis_rawWorld = transform_R.T @ emitter_window['light_world_total3d_centeraxis'][1]


# ====== convert SG to global envmap (global RAW coords)
emitter_prop_total3d = emitters_prop_dict_representation_dict[random_id_chosen]['emitter_prop_total3d']

envScale = emitter_prop_total3d['envScale']

weight_SG = emitter_prop_total3d['intensity']
# weight_SG = [x / light_dat['envScale'] for x in weight_SG]
lamb_SG = emitter_prop_total3d['lamb']

light_axis_world_SG = np.array([light_axis_rawWorld[2], -light_axis_rawWorld[0], light_axis_rawWorld[1]])
cos_theta = light_axis_world_SG[2]
theta_SG = np.arccos(cos_theta) # [0, pi]
cos_phi = light_axis_world_SG[0] / np.sin(theta_SG)
sin_phi = light_axis_world_SG[1] / np.sin(theta_SG)
phi_SG = np.arctan2(sin_phi, cos_phi)
assert phi_SG >= -np.pi and phi_SG <= np.pi
ambient_SG = np.array([0., 0., 0.])

print(theta_SG)
print(phi_SG)
print(lamb_SG)
print(weight_SG)
print(ambient_SG)

envOptim = SGOptim.SGEnvOptimSky(weightValue=np.zeros(3,), thetaValue=0., phiValue=0., ambientValue=np.zeros(3,), isCuda=True)
recImg = envOptim.renderSG(torch.tensor(theta_SG).reshape((1, 1, 1, 1, 1, 1)).cuda(), torch.tensor(phi_SG).reshape((1, 1, 1, 1, 1, 1)).cuda(), \
                  torch.tensor(lamb_SG).reshape((1, 1, 1, 1, 1)).cuda(), torch.tensor(weight_SG).reshape((1, 1, 3, 1, 1)).cuda(), \
                           ambient=torch.tensor(ambient_SG).reshape((1, 1, 3, 1, 1)).cuda())
recImg_gen = recImg.squeeze().cpu().numpy().transpose(1, 2, 0)

# recIm_local_uint8, _ = to_nonhdr(recIm.squeeze().transpose(1, 2, 0), scale=im_envmap_ori_scale)


['9HSBL', 'QICJ8']
[0.4274]
[-1.168]
[[[6.1417]]]
[3.709547281265259, 3.8425681591033936, 4.323965549468994]
[0. 0. 0.]


In [149]:
env_scale, hdr_scale

(119.1801, 0.1263157894736842)

In [152]:
from pathlib import Path
import sys
sys.path.insert(0, '/home/ruizhu/Documents/Projects/Total3DUnderstanding/utils_OR')
from utils_OR.utils_OR_xml import get_XML_root, parse_XML_for_shapes
import random

# main_xml_file = Path(xml_path) / sequence['sub_name'] / sequence['scene_name'] / ('%s.xml'%sequence['meta_name'])
# root = get_XML_root(main_xml_file)
# random_seed = random.randint(0, 10000)
# shape_list, emitters_list = parse_XML_for_shapes(root, root_uv_mapped, if_return_emitters=True, random_seed=random_seed)
# assert emitters_list[0]['emitter_prop']['if_env'] == True
# envmap_path = emitters_list[0]['emitter_prop']['emitter_filename']
envmap_path = emitter_prop_total3d['envMapPath']
envmap_path = envmap_path.replace('../../../../../EnvDataset/', envmap_root)

envmap_scale = emitter_prop_total3d['envScale']

from utils_OR.utils_OR_imageops import loadHdr_simple, to_nonhdr
im_envmap_ori = loadHdr_simple(envmap_path)
im_envmap_ori_uint8, im_envmap_ori_scale = to_nonhdr(im_envmap_ori)
print(im_envmap_ori_scale)


from utils_OR.utils_OR_imageops import loadHdr, loadImage, scaleHdr, load_OR_hdr_uint8

lightId = int(light_path.split('/')[-1].split('.')[0].replace('light0ambient', ''))

img_path_halfenvmap = Path(xml_path) / sequence['sub_name'] / sequence['scene_name'] / \
('%s-light%d_1.rgbe' % (sequence['meta_name'].replace('mainDiffMat', 'main'), lightId ))
print(img_path_halfenvmap)
extra_scale = 1.

im_uint8_halfenvmap, (im_halfenvmap, _, _) = load_OR_hdr_uint8(img_path_halfenvmap, scale=im_envmap_ori_scale * extra_scale)
recImg_gen_uint8, _ = to_nonhdr(recImg_gen, scale=im_envmap_ori_scale * extra_scale)
# im_uint8_fullenvmap, (im_fullenvmap, _, _) = load_OR_hdr_uint8(img_path_halfenvmap, scale=im_envmap_ori_scale * extra_scale)



plt.figure(figsize=(15, 8))
ax = plt.subplot(221)
plt.imshow(im_uint8_halfenvmap)
ax.set_title('ori half env map')

ax = plt.subplot(222)
plt.imshow(im_envmap_ori_uint8)
ax.set_title('Ori env map (global RAW coords)')

ax = plt.subplot(224)
plt.imshow(recImg_gen_uint8)
ax.set_title('render SG env map (global RAW coords)')


plt.show()

/home/ruizhu/Documents/data/EnvDataset/84L.hdr 8.5
0.15652173913043477
/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes/xml/scene0385_01/mainDiffLight-light4_1.rgbe


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# [1] Visualize per-pixel lighting & geometry

In [159]:
lighting_envmaps_extra_scale_VIS = 0.1

## GT lighting (env_scale bakced in; hdr_scale is not)

In [160]:
%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


from utils_notebook import *

envHeight = 8
envWidth = 16
envRow = 120
envCol = 160
SGNum = 12

im_width = 320
im_height = 240
bn = 1

from pathlib import Path
# meta_split, scene_name, png_file_name = ('main_xml1', 'scene0552_00', 'im_1.png')
png_file_name = 'im_%d.png'%withinsequence_id
# meta_split, scene_name, png_file_name = ('mainDiffMat_xml1', 'scene0695_03', 'im_26.png')

root_path = Path('/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/DatasetCreation/')
env_path = root_path / meta_split / scene_name / (png_file_name.replace('im_', 'imenv_').replace('.png', '.hdr'))

# change to direct lighting; comment 3 lines to use original envmap lighting
# env_path = str(env_path).replace('imenv', 'imenvDirect')
# envRow = 120 // 4
# envCol = 160 // 4

envmaps, envmapsInd = loadEnvmap(str(env_path))

H_grid, W_grid, h, w = envmaps.shape[1:]
downsize_ratio = 10
assert H_grid % downsize_ratio == 0
assert W_grid % downsize_ratio == 0
xx, yy = np.meshgrid(np.arange(0, H_grid, downsize_ratio), np.arange(0, W_grid, downsize_ratio))

a = envmaps[:, xx.T, yy.T, :, :] * hdr_scale * lighting_envmaps_extra_scale_VIS
a = np.clip(a**(1.0/2.2), 0., 1.)


print(a.shape)
bb = a.transpose(1, 3, 2, 4, 0).reshape(H_grid*h//downsize_ratio, W_grid*w//downsize_ratio, 3)


import matplotlib.pyplot as plt
hdr_path = root_path / meta_split / scene_name / (png_file_name.replace('.png', '.hdr'))
# im = loadHdr(str(hdr_path))
im_hdr_ori = loadHdr_simple(str(hdr_path))
im_hdr_ori_uint8, im_hdr_ori_scale = to_nonhdr(im_hdr_ori)

normal_path = str(root_path / meta_split / scene_name / png_file_name).replace('im_', 'imnormal_').replace('DiffLight', '')
# normalize the normal vector so that it will be unit length
normal = loadImage(str(normal_path) )
normal = normal / np.sqrt(np.maximum(np.sum(normal * normal, axis=0), 1e-5) )[np.newaxis, :]
# normalPred = torch.from_numpy(normal).cuda().unsqueeze(0)

depth_path = str(root_path / meta_split / scene_name / (png_file_name.replace('im_', 'imdepth_').replace('png', 'dat'))).replace('DiffLight', '').replace('DiffMat', '')
# Read depth
depth = loadBinary(depth_path).squeeze()

f = 577.8708 / 2.
u0 = im_width / 2.
v0 = im_height / 2.

uu, vv = np.meshgrid(range(im_width), range(im_height))
z = -depth
x = - (uu - u0) / f * z
y = (vv - v0) / f * z
points = np.stack([x.squeeze(), y.squeeze(), z.squeeze()])

plt.figure(figsize=(12, 8))
plt.subplot(221)
plt.hist(depth.reshape(-1), 100)
plt.subplot(222)
plt.imshow(depth)
plt.colorbar()
plt.subplot(224)
mask = depth > 0.1
plt.imshow(mask)
plt.subplot(223)
normal_vis = normal.transpose(1, 2, 0)
normal_vis = (normal_vis + 1.)/2.
plt.imshow(normal_vis)
plt.show()

plt.figure(figsize=(15, 8))
plt.imshow(bb)
plt.show()

>>>>loadEnvmap /newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/DatasetCreation/mainDiffLight_xml/scene0385_01/imenv_32.hdr
(1920, 5120, 3)
(3, 12, 16, 8, 16)
/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/DatasetCreation/mainDiffLight_xml/scene0385_01/im_32.hdr 84.5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## EST lighting (env_scale bakced in; hdr_scale is not)

In [161]:
# read EST geometry + lighting
    
normalPred_lightAccu, depthPred_lightAccu, envmapsPredImage_lightAccu = results_emitter_input_pickle_dict['normalPred_lightAccu'], results_emitter_input_pickle_dict['depthPred_lightAccu'], results_emitter_input_pickle_dict['envmapsPredImage_lightAccu']
print(normalPred_lightAccu.shape, depthPred_lightAccu.shape, envmapsPredImage_lightAccu.shape)
print(normal.shape, depth.shape, envmaps.shape)

plt.figure(figsize=(10, 8))
plt.subplot(221)
normal_vis = normal.transpose(1, 2, 0)
normal_vis = (normal_vis + 1.)/2.
plt.imshow(normal_vis)
plt.subplot(222)
normal_vis = normalPred_lightAccu.transpose(1, 2, 0)
normal_vis = (normal_vis + 1.)/2.
plt.imshow(normal_vis)
plt.subplot(223)
plt.imshow(depth)
plt.colorbar()
plt.subplot(224)
plt.imshow(depthPred_lightAccu)
plt.colorbar()
plt.show()

# lighting_envmaps_scale = 0.1 # [!!! set this!!!]

H_grid, W_grid, h, w = envmapsPredImage_lightAccu.shape[1:]
downsize_ratio = 10
assert H_grid % downsize_ratio == 0
assert W_grid % downsize_ratio == 0
xx, yy = np.meshgrid(np.arange(0, H_grid, downsize_ratio), np.arange(0, W_grid, downsize_ratio))

a = envmapsPredImage_lightAccu[:, xx.T, yy.T, :, :] * lighting_envmaps_extra_scale_VIS
a = np.clip(a**(1.0/2.2), 0., 1.)


print(a.shape)
b = a.transpose(1, 3, 2, 4, 0).reshape(H_grid*h//downsize_ratio, W_grid*w//downsize_ratio, 3)


import matplotlib.pyplot as plt
plt.figure(figsize=(15, 8))
plt.imshow(b)
plt.show()

(3, 240, 320) (240, 320) (3, 120, 160, 8, 16)
(3, 240, 320) (240, 320) (3, 120, 160, 8, 16)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3, 12, 16, 8, 16)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# [3] visualize model outputs

In [156]:
env_scale, hdr_scale, im_envmap_ori_scale

(119.1801, 0.1263157894736842, 0.15652173913043477)

### (1/2) from network with GT input

In [162]:
B = 2
depth_torch = torch.from_numpy(depth).cuda().float().unsqueeze(0).repeat(B, 1, 1).contiguous() # (B, 240, 320)
normal_torch = torch.from_numpy(normal).cuda().float().unsqueeze(0).repeat(B, 1, 1, 1).contiguous() # (B, 3, 240, 320)
envmaps_torch = torch.from_numpy(envmaps).cuda().float().unsqueeze(0).repeat(B, 1, 1, 1, 1, 1).contiguous() # (B, 3, 120, 160, 8, 16)
envmaps_torch = envmaps_torch * hdr_scale # [!!!!!! IMPORTANT] to recover true lightign scale of the sample
cam_K_torch = torch.from_numpy(sequence['camera']['K']).cuda().float().unsqueeze(0).repeat(B, 1, 1).contiguous()
layout_torch = torch.from_numpy(gt_layout).cuda().float().unsqueeze(0).repeat(B, 1, 1).contiguous()
cam_R_gt_torch = torch.from_numpy(cam_R_gt).cuda().float().unsqueeze(0).repeat(B, 1, 1).contiguous()

input_dict = {'normalPred_lightAccu':normal_torch, 'depthPred_lightAccu': depth_torch, 'envmapsPredImage_lightAccu': envmaps_torch, 'cam_K': cam_K_torch, 'cam_R': cam_R_gt_torch, 'layout': layout_torch}
output_path = '/home/ruizhu/Downloads/tmp.pickle'


import sys
sys.path.insert(0, '/home/ruizhu/Documents/Projects/semanticInverse/train')
import torch
%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from models_def.models_layout_emitter_lightAccu import emitter_lightAccu

# scatterHeight = 8
# scatterWidth = 16
scatterHeight = 16
scatterWidth = 32

lightAccuNet = emitter_lightAccu(params={'im_width': 320, 'im_height': 240, 'envmapHeight': 512, 'envmapWidth': 1024}, scatterHeight=scatterHeight, scatterWidth=scatterWidth).cuda()

input_dict = {'normalPred_lightAccu':normal_torch, 'depthPred_lightAccu': depth_torch, 'envmapsPredImage_lightAccu': envmaps_torch, 'cam_K': cam_K_torch, 'cam_R': cam_R_gt_torch, 'layout': layout_torch}

return_dict_lightAccu = lightAccuNet(input_dict) # 1.4Gi, [2, 3, 384, 120, 160]) [2, 384, 3]
# print(X.shape, envmap_vis_mean_.shape)

envmap_lightAccu = return_dict_lightAccu['envmap_lightAccu'] * lighting_envmaps_extra_scale_VIS # [3, 384, 120, 160]
envmap_lightAccu = envmap_lightAccu[0].detach().cpu().numpy()

envmap_lightAccu_mean = return_dict_lightAccu['envmap_lightAccu_mean'] * lighting_envmaps_extra_scale_VIS # (384, 3)
envmap_lightAccu_mean = envmap_lightAccu_mean[0].detach().cpu().numpy()

color_array_full = np.clip(envmap_lightAccu_mean**(1.0/2.2), 0., 1.)
# color_array_full = np.clip(envmap_lightAccu_mean, 0., 1.)
print(color_array_full.shape)
plt.figure(figsize=(8, 2))
plt.subplot(121)
plt.hist(envmap_lightAccu_mean, 100)
plt.subplot(122)
plt.hist(color_array_full, 100)
plt.title('color_array_full')
plt.show()

(384, 3)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [163]:
from models_def.models_layout_emitter_lightAccuScatter import decoder_layout_emitter_lightAccuScatter_UNet_V3

lightDecoderNet = decoder_layout_emitter_lightAccuScatter_UNet_V3(other_flags=['use_weighted_axis'], scatterHeight=scatterHeight, scatterWidth=scatterWidth).cuda()
# envmap_lightAccu = return_dict['envmap_lightAccu_mean'].view(-1, 6, 8, 8, 3).permute(0, 1, 4, 2, 3)
# print(envmap_lightAccu.shape)

scattered_light, emitter_local_xyz_trans = lightAccuNet.scatter_light_to_hemisphere(return_dict_lightAccu)

# ---- Get params to transform cell_axis from LightNet coords -> Total3D coords
Total3D_to_LightNet_transform_params = return_dict_lightAccu['Total3D_to_LightNet_transform_params']
cam_R_transform_matrix_pre, post_transform_matrix = Total3D_to_LightNet_transform_params['cam_R_transform_matrix_pre'], Total3D_to_LightNet_transform_params['post_transform_matrix']
inv_post_transform_matrix_expand = post_transform_matrix.unsqueeze(1).unsqueeze(1).unsqueeze(1).transpose(-1, -2)
inv_inv_cam_R_transform_matrix_pre_expand = cam_R_transform_matrix_pre.unsqueeze(1).unsqueeze(1).unsqueeze(1)
transform_params_LightNet2Total3D = {'inv_post_transform_matrix_expand': inv_post_transform_matrix_expand, 'inv_inv_cam_R_transform_matrix_pre_expand': inv_inv_cam_R_transform_matrix_pre_expand}

T_LightNet2Total3D_rightmult = transform_params_LightNet2Total3D['inv_post_transform_matrix_expand'] @ transform_params_LightNet2Total3D['inv_inv_cam_R_transform_matrix_pre_expand']

# ---- Get the dir meshgrid outside (abs/relative) for all cells
emitter_outdirs_meshgrid_global_lightNet = lightAccuNet.get_emitter_outdirs_meshgrid(emitter_local_xyz_trans)
grid_size = 8
normal_outside_lightNet = - return_dict_lightAccu['normal_array_lightNet'].unsqueeze(2).unsqueeze(2).unsqueeze(2) # [B, 6, 1, 1, 1, 3]
normal_outside_lightNet = normal_outside_lightNet.repeat(1, 1, grid_size**2, 1, 1, 1) # normal inside --negative--> normal outside # [B, ngrids(6*8*8, 1, 1, 3]
normal_outside_lightNet = normal_outside_lightNet.view(normal_outside_lightNet.shape[0], 6*grid_size*grid_size, 1, 1, 3) # [B, ngrids, 1, 1, 3]

emitter_outdirs_meshgrid_global_lightNet_outside = - emitter_outdirs_meshgrid_global_lightNet # flip the hemisphere dirs so that they point to the outside of the wall

# ---- LightNet -> Total3D (dirs & normal outside)
emitter_outdirs_meshgrid_Total3D_outside = (emitter_outdirs_meshgrid_global_lightNet_outside.unsqueeze(-2) @ T_LightNet2Total3D_rightmult).squeeze(-2) # [B, 384, 8, 16, 3]
normal_outside_Total3D = (normal_outside_lightNet.unsqueeze(-2) @ T_LightNet2Total3D_rightmult).squeeze(-2) # [B, 384, 8, 16, 3]            

return_dict_layout_emitter = lightDecoderNet(scattered_light, emitter_outdirs_meshgrid_Total3D_outside)

# convert to NUMPY
scattered_light = scattered_light[0].cpu().detach().numpy() * lighting_envmaps_extra_scale_VIS
emitter_outdirs_meshgrid_Total3D_outside = emitter_outdirs_meshgrid_Total3D_outside[0].cpu().detach().numpy()
normal_outside_Total3D = normal_outside_Total3D[0].cpu().detach().numpy()

# wall_idx, cell_i, cell_j = 2, 3, 3

cell_idx = wall_idx * grid_size**2 + cell_i * grid_size + cell_j

# %matplotlib widget

meshgrid_dir_map = emitter_outdirs_meshgrid_Total3D_outside[cell_idx]
# np.linalg.norm(dir_map, axis=2)
gt_dir_map = emitter_window['light_world_total3d_centeraxis'][1].flatten()[np.newaxis, np.newaxis, :]
np.linalg.norm(gt_dir_map, axis=2)
l2_dist_map = np.linalg.norm(gt_dir_map - meshgrid_dir_map, axis=2)
l2_dist_map = -l2_dist_map
l2_dist_map = l2_dist_map - np.amin(l2_dist_map)
l2_dist_map = l2_dist_map / np.amax(l2_dist_map)

fig = plt.figure(figsize=(15, 5))
fig.suptitle('GT scatter map and heatmap [val-%d][%d-%d-%d]'%(pickle_id, wall_idx, cell_i, cell_j))

plt.subplot(121)
# plt.imshow(scattered_light[cell_idx])
plt.imshow(scattered_light[cell_idx]**(1./2.2))
plt.subplot(122)
plt.imshow(l2_dist_map)
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [164]:

for wall_cell_idx in wall_cell_idxes:
    wall_idx_tmp, cell_i_tmp, cell_j_tmp = wall_cell_idx[0], wall_cell_idx[1], wall_cell_idx[2]
    cell_idx_tmp = wall_idx_tmp * grid_size**2 + cell_i_tmp * grid_size + cell_j_tmp


    meshgrid_dir_map = emitter_outdirs_meshgrid_Total3D_outside[cell_idx]
    # np.linalg.norm(dir_map, axis=2)
    gt_dir_map = emitter_window['light_world_total3d_centeraxis'][1].flatten()[np.newaxis, np.newaxis, :]
    np.linalg.norm(gt_dir_map, axis=2)
    l2_dist_map = np.linalg.norm(gt_dir_map - meshgrid_dir_map, axis=2)
    l2_dist_map = -l2_dist_map
    l2_dist_map = l2_dist_map - np.amin(l2_dist_map)
    l2_dist_map = l2_dist_map / np.amax(l2_dist_map)

    fig = plt.figure(figsize=(15, 5))
    fig.suptitle('GT scatter map and heatmap [val-%d][%d-%d-%d]'%(pickle_id, wall_idx_tmp, cell_i_tmp, cell_j_tmp))

    plt.subplot(121)
    # plt.imshow(scattered_light[cell_idx])
    plt.imshow(np.clip(scattered_light[cell_idx_tmp]**(1./2.2), 0., 1.))
    plt.subplot(122)
    plt.imshow(l2_dist_map)
    plt.colorbar()
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/ruizhu/anaconda3/envs/semanticInverse/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  from ipykernel import kernelapp as app


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### (2/2) read from saved est. result pickles

In [138]:
scatter_light_path = '/home/ruizhu/Documents/Projects/semanticInverse/Summary_vis/%s/results_emitter_%d.pickle'%(results_folder, pickle_id)
with open(scatter_light_path, 'rb') as f:
    scatter_light_dat = pickle.load(f)
    
for key in scatter_light_dat:
    print(key, scatter_light_dat[key].shape)
    
est_light_scale = im_envmap_ori_scale

scattered_light = scatter_light_dat['scattered_light'] * lighting_envmaps_extra_scale_VIS # (384, 8, 16, 3)
envmap_lightAccu = scatter_light_dat['envmap_lightAccu'] * lighting_envmaps_extra_scale_VIS # , (3, 384, 120, 160)
envmap_lightAccu_mean = scatter_light_dat['envmap_lightAccu_mean'] * lighting_envmaps_extra_scale_VIS # (6, 3, 8, 8)
points_sampled_mask_expanded = scatter_light_dat['points_sampled_mask_expanded']

color_array_full = envmap_lightAccu_mean.transpose(0, 2, 3, 1)

# wall_idx, cell_i, cell_j = 2, 3, 3

cell_idx = wall_idx * grid_size**2 + cell_i * grid_size + cell_j

normal_outside_Total3D = scatter_light_dat['normal_outside_Total3D']
emitter_outdirs_meshgrid_Total3D_outside = scatter_light_dat['emitter_outdirs_meshgrid_Total3D_outside'] + normal_outside_Total3D
cell_axis_weights = scatter_light_dat['cell_axis_weights'].squeeze() # (384, 8, 16)
emitter_cell_axis_abs_est = scatter_light_dat['emitter_cell_axis_abs_est']
emitter_cell_axis_abs_gt = scatter_light_dat['emitter_cell_axis_abs_gt']

%matplotlib widget

fig = plt.figure(figsize=(15, 5))
fig.suptitle('EST scatter map and heatmap [val-%d][%d-%d-%d]'%(pickle_id, wall_idx, cell_i, cell_j))

plt.subplot(121)
# plt.imshow(scattered_light[cell_idx] / env_scale)
plt.imshow(scattered_light[cell_idx]**(1./2.2))
plt.subplot(122)
plt.imshow(cell_axis_weights[cell_idx])
plt.show()

envmap_lightAccu_mean (6, 3, 8, 8)
points_sampled_mask_expanded (1, 1, 120, 160)
cell_normal_outside_label (6, 8, 8, 3)
emitter_cell_axis_abs_est (6, 64, 3)
emitter_cell_axis_abs_gt (6, 64, 3)
window_mask (6, 64)
envmap_lightAccu (3, 384, 120, 160)
scattered_light (384, 8, 16, 3)
emitter_outdirs_meshgrid_Total3D_outside (384, 8, 16, 3)
normal_outside_Total3D (384, 1, 1, 3)
cell_axis_weights (384, 8, 16, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


# [5] vis in Total3D coords
RUN ``new UNet decoder [V3]`` block first!

https://i.imgur.com/prxdna3.png

In [67]:
(wall_idx, cell_i, cell_j)

(4, 2, 4)

In [66]:
grid_size = 8

lightnet_array_GT = color_array_full.reshape(6, grid_size, grid_size, -1)
lightnet_array_GT = lightnet_array_GT * 5
lightnet_array_GT = np.clip(lightnet_array_GT**(1./2.2), 0., 1.)

%matplotlib widget

scene_box = Box(rgb_img, None, sequence['camera']['K'], cam_R_gt, None, gt_layout, None, gt_boxes_dict, None, 'GT', None, dataset=dataset, OR='OR45', \
                emitters_obj_list=emitters_obj_list, emitter2wall_assign_info_list=emitter2wall_assign_info_list, \
                emitter_cls_prob_PRED=np.random.random((6, grid_size**2)), emitter_cls_prob_GT=grid_prob, cell_info_grid_GT=cell_info_grid_GT, grid_size=grid_size, \
                paths={'color_file': '../train/data/openrooms/total3D_colors/OR4X_mapping_catInt_to_RGB_light.pkl'})

fig_3d, ax_3d_scene = scene_box.draw_3D_scene_plt('GT', hide_cells=False, if_show_objs=False, scale_emitter_length=2.)
ax_3d_scene[1] = fig_3d.add_subplot(122, projection='3d')
scene_box.draw_3D_scene_plt('GT', fig_or_ax=[ax_3d_scene[1], ax_3d_scene[0]], hide_cells=False, if_show_objs=False, scale_emitter_length=2., if_print_log=True)

scene_box.draw_all_cells(ax_3d_scene[1], gt_layout, lightnet_array_GT=lightnet_array_GT, alpha=1., highlight_cells=[(wall_idx, cell_i, cell_j)])

az = 112
elev = 19
ax_3d_scene[1].view_init(elev=elev, azim=az)
ax_3d_scene[0].view_init(elev=elev, azim=az)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## (1/2) read from result pickles

In [80]:
from SimpleLayout.utils_SL_vis import Arrow3D

# draw GT/EST dirs
for cell_info in cell_info_grid_GT:
    cell_center = cell_info['cell_center'].flatten()

    if cell_info['obj_type']=='window':
        
        wall_idx_tmp, cell_i_tmp, cell_j_tmp = cell_info['wallidx_i_j'][0], cell_info['wallidx_i_j'][1], cell_info['wallidx_i_j'][2]

#         emitter_cell_axis_abs_single = emitter_cell_axis_abs_gt[wall_idx_tmp, cell_i_tmp*grid_size+cell_j_tmp]
        emitter_cell_axis_abs_single = emitter_cell_axis_abs_est[wall_idx, cell_i_tmp*grid_size+cell_j_tmp]

        emitter_cell_axis_abs_single_end = emitter_cell_axis_abs_single * 3 + cell_center

        a = Arrow3D([cell_center[0], emitter_cell_axis_abs_single_end[0]], [cell_center[1], emitter_cell_axis_abs_single_end[1]], [cell_center[2], emitter_cell_axis_abs_single_end[2]], mutation_scale=20,
            lw=0.5, arrowstyle="-|>", facecolor='y')
        ax_3d_scene[0].add_artist(a)
        
az = 111
elev = -42
ax_3d_scene[1].view_init(elev=elev, azim=az)
ax_3d_scene[0].view_init(elev=elev, azim=az)


### hemisphere from GT/EST

In [81]:
# Draw emtter hemisphere (directions + scattered light color) acquired from the network in Total3D coords
for cell_info in cell_info_grid_GT:
    cell_center = cell_info['cell_center'].flatten()

    if cell_info['obj_type']=='window':
        
        if cell_info['wallidx_i_j'] != (wall_idx, cell_i, cell_j):
            continue

        for ii in range(8):
            for jj in range(16):
                # EST saved
                emitter_dir_single_show = emitter_outdirs_meshgrid_Total3D_outside[cell_idx, ii, jj]
                cell_normal_outside = scatter_light_dat['cell_normal_outside_label'][wall_idx, cell_i, cell_j]
                if np.amax(np.abs(normal_outside_Total3D[cell_idx].flatten()-cell_normal_outside)) > 1e-4:
                    print(normal_outside_Total3D[cell_idx].flatten(), cell_normal_outside)
                    assert False
                emitter_dir_end = cell_center.flatten() + emitter_dir_single_show * 2.

                color = np.clip(scattered_light[cell_idx, ii, jj].flatten(), 0., 1.)
#                 color = 'k'

#                 a = Arrow3D([cell_center[0], emitter_dir_end[0]], [cell_center[1], emitter_dir_end[1]], [cell_center[2], emitter_dir_end[2]], mutation_scale=20,
#                     lw=0.5, arrowstyle="-|>", facecolor=color)
#                 ax_3d_scene[0].add_artist(a)

                ax_3d_scene[0].scatter3D(emitter_dir_end[0], emitter_dir_end[1], emitter_dir_end[2], color=color, edgecolor='k', s = 50)


        window_normal_outside_end = cell_normal_outside * 3 + cell_center

        a = Arrow3D([cell_center[0], window_normal_outside_end[0]], [cell_center[1], window_normal_outside_end[1]], [cell_center[2], window_normal_outside_end[2]], mutation_scale=20,
            lw=0.5, arrowstyle="Fancy", facecolor='y')
        ax_3d_scene[0].add_artist(a)

                

## (2/2) acquire from network with GT inputs

In [85]:
verts_center_transformed_LightNet = return_dict_lightAccu['verts_center_lightNet'][0].cpu().numpy()
verts_transformed_LightNet = return_dict_lightAccu['verts_lightNet'][0].cpu().numpy()
basis_1_array_transformed_LightNet = return_dict_lightAccu['basis_1_array_lightNet'][0].cpu().numpy()
basis_2_array_transformed_LightNet = return_dict_lightAccu['basis_2_array_lightNet'][0].cpu().numpy()
origin_0_array_transformed_LightNet = return_dict_lightAccu['origin_0_array_lightNet'][0].cpu().numpy()
normal_inside_array_transformed_LightNet = return_dict_lightAccu['normal_array_lightNet'][0].cpu().numpy()

basis_1 = basis_1_array_transformed_LightNet[wall_idx]
basis_2 = basis_2_array_transformed_LightNet[wall_idx]
normal_inside = normal_inside_array_transformed_LightNet[wall_idx]

T_LightNet2Total3D_rightmult_np = T_LightNet2Total3D_rightmult[0].squeeze().cpu().numpy()


basis_1_total3D = ((basis_1.reshape(1, 3) @ T_LightNet2Total3D_rightmult_np).T).flatten()
basis_1_total3D = - basis_1_total3D # FLIPPED to outside!!!
basis_2_total3D = ((basis_2.reshape(1, 3) @ T_LightNet2Total3D_rightmult_np).T).flatten()
basis_2_total3D = - basis_2_total3D # FLIPPED to outside!!!
normal_inside_total3D = ((normal_inside.reshape(1, 3) @ T_LightNet2Total3D_rightmult_np).T).flatten()
normal_outside_total3D = - normal_inside_total3D # FLIPPED to outside!!!

# emitter local axes in (RAW->)Total3D
a = Arrow3D([cell_center[0], cell_center[0]+basis_1_total3D[0]], [cell_center[1], cell_center[1]+basis_1_total3D[1]], [cell_center[2], cell_center[2]+basis_1_total3D[2]], mutation_scale=20,
            lw=0.5, arrowstyle="-|>", facecolor='r')
ax_3d_scene[0].add_artist(a)
a = Arrow3D([cell_center[0], cell_center[0]+basis_2_total3D[0]], [cell_center[1], cell_center[1]+basis_2_total3D[1]], [cell_center[2], cell_center[2]+basis_2_total3D[2]], mutation_scale=20,
            lw=0.5, arrowstyle="-|>", facecolor='g')
ax_3d_scene[0].add_artist(a)
a = Arrow3D([cell_center[0], cell_center[0]+normal_outside_total3D[0]], [cell_center[1], cell_center[1]+normal_outside_total3D[1]], [cell_center[2], cell_center[2]+normal_outside_total3D[2]], mutation_scale=20,
            lw=0.5, arrowstyle="-|>", facecolor='b')
ax_3d_scene[0].add_artist(a)


In [88]:
transform_R_RAW2Total3D, transform_t_RAW2Total3D = transform_to_total3d_coords_dict['transform_R'], transform_to_total3d_coords_dict['transform_t']

envAxis_x_total3D = (transform_R_RAW2Total3D @ emitter_prop_total3d['envAxis_x'].reshape((3,1))).flatten()
envAxis_y_total3D = (transform_R_RAW2Total3D @ emitter_prop_total3d['envAxis_y'].reshape((3,1))).flatten()
envAxis_z_total3D = (transform_R_RAW2Total3D @ emitter_prop_total3d['envAxis_z'].reshape((3,1))).flatten()

# SG local axes in (RAW->)Total3D
a = Arrow3D([cell_center[0], cell_center[0]+envAxis_x_total3D[0]], [cell_center[1], cell_center[1]+envAxis_x_total3D[1]], [cell_center[2], cell_center[2]+envAxis_x_total3D[2]], mutation_scale=20,
            lw=0.5, arrowstyle="Fancy", facecolor='r')
ax_3d_scene[0].add_artist(a)
a = Arrow3D([cell_center[0], cell_center[0]+envAxis_y_total3D[0]], [cell_center[1], cell_center[1]+envAxis_y_total3D[1]], [cell_center[2], cell_center[2]+envAxis_y_total3D[2]], mutation_scale=20,
            lw=0.5, arrowstyle="Fancy", facecolor='g')
ax_3d_scene[0].add_artist(a)
a = Arrow3D([cell_center[0], cell_center[0]+envAxis_z_total3D[0]], [cell_center[1], cell_center[1]+envAxis_z_total3D[1]], [cell_center[2], cell_center[2]+envAxis_z_total3D[2]], mutation_scale=20,
            lw=0.5, arrowstyle="Fancy", facecolor='b')
ax_3d_scene[0].add_artist(a)


KeyError: 'envAxis_z'

# [5] sample ori/SG envmaps

In [144]:
emitter_dirs_Total3D = emitter_outdirs_meshgrid_Total3D_outside[cell_idx][:, :, :, np.newaxis] # [8, 16, 3]

transform_R_RAW2Total3D, transform_t_RAW2Total3D = transform_to_total3d_coords_dict['transform_R'], transform_to_total3d_coords_dict['transform_t']

transform_R_Total3D2RAW = transform_R_RAW2Total3D.T[np.newaxis, np.newaxis, :]
print(transform_R_Total3D2RAW.shape, emitter_dirs_Total3D.shape)
emitter_dirs_RAW = transform_R_Total3D2RAW @ emitter_dirs_Total3D
print(emitter_dirs_RAW.shape)
emitter_dirs_RAW = emitter_dirs_RAW.squeeze()

(1, 1, 3, 3) (8, 16, 3, 1)
(8, 16, 3, 1)


In [145]:
light_axis_world_SG = np.stack([emitter_dirs_RAW[:, :, 2], -emitter_dirs_RAW[:, :, 0], emitter_dirs_RAW[:, :, 1]], -1)
print(light_axis_world_SG.shape)
cos_theta = light_axis_world_SG[:, :, 2]
theta_SG = np.arccos(cos_theta) # [0, pi]
cos_phi = light_axis_world_SG[:, :, 0] / np.sin(theta_SG)
sin_phi = light_axis_world_SG[:, :, 1] / np.sin(theta_SG)
phi_SG = np.arctan2(sin_phi, cos_phi)
assert np.all(phi_SG >= -np.pi)
assert np.all(phi_SG <= np.pi)

az = phi_SG
az.shape
envmap_H, envmap_W = im_envmap_ori_uint8.shape[:2]
# az_pix = (phi_SG / np.pi / 2. + 0.5) * envmap_W - 0.5 # [2, 384, 120, 160] # ideally np.arange(envWidth)
# el_pix = theta_SG / np.pi * 2. * envmap_H - 0.5 # [2, 384, 120, 160] # ideally np.arange(envHeight)
uu_pix = (phi_SG + np.pi) / (2 * np.pi) * envmap_W
vv_pix = theta_SG / np.pi * envmap_H

uu_pix = np.clip(np.round(uu_pix).astype(np.int), 0, envmap_W-1)
vv_pix = np.clip(np.round(vv_pix).astype(np.int), 0, envmap_H-1)

envmap_sampled = im_envmap_ori_uint8[vv_pix, uu_pix]

plt.figure()
plt.imshow(envmap_sampled)
plt.show()

(8, 16, 3)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## from the sampler in the network

In [65]:
cell_idx

155

In [66]:
emitter_outdirs_meshgrid_Total3D_outside[155]

array([[[ 0.9362,  0.0961, -0.3382],
        [ 0.9248,  0.0815, -0.3716],
        [ 0.9161,  0.0545, -0.3972],
        [ 0.9114,  0.0191, -0.4111],
        [ 0.9114, -0.0191, -0.4111],
        [ 0.9161, -0.0545, -0.3972],
        [ 0.9248, -0.0815, -0.3716],
        [ 0.9362, -0.0961, -0.3382],
        [ 0.9485, -0.0961, -0.302 ],
        [ 0.9598, -0.0815, -0.2685],
        [ 0.9685, -0.0545, -0.2429],
        [ 0.9732, -0.0191, -0.229 ],
        [ 0.9732,  0.0191, -0.229 ],
        [ 0.9685,  0.0545, -0.2429],
        [ 0.9598,  0.0815, -0.2685],
        [ 0.9485,  0.0961, -0.302 ]],

       [[ 0.8879,  0.2847, -0.3614],
        [ 0.8542,  0.2414, -0.4605],
        [ 0.8285,  0.1613, -0.5363],
        [ 0.8145,  0.0566, -0.5773],
        [ 0.8145, -0.0566, -0.5773],
        [ 0.8285, -0.1613, -0.5363],
        [ 0.8542, -0.2414, -0.4605],
        [ 0.8879, -0.2847, -0.3614],
        [ 0.9243, -0.2847, -0.2541],
        [ 0.958 , -0.2414, -0.1551],
        [ 0.9837, -0.1613, -0.0792],

In [147]:
emitter_outdirs_meshgrid_Total3D_outside.shape, transform_R_RAW2Total3D.shape, transform_t_RAW2Total3D.shape, im_envmap_ori.shape

input_dict_sampler = {'emitter_outdirs_meshgrid_Total3D_outside': torch.from_numpy(emitter_outdirs_meshgrid_Total3D_outside).cuda().unsqueeze(0).float(), \
                     'transform_R_RAW2Total3D': torch.from_numpy(transform_R_RAW2Total3D).cuda().unsqueeze(0).float(), \
                     'im_envmap_ori': torch.from_numpy(im_envmap_ori).cuda().unsqueeze(0).float()}

lightAccuNet = emitter_lightAccu(params={'im_width': 320, 'im_height': 240, 'envmapHeight': 512, 'envmapWidth': 1024}, scatterHeight=scatterHeight, scatterWidth=scatterWidth)

im_envmap_sampled = lightAccuNet.sample_envmap(input_dict_sampler)

plt.figure()
envmap_sampled_network = im_envmap_sampled.cpu().numpy()[0, :, cell_idx, :, :].transpose(1, 2, 0)
plt.imshow(envmap_sampled_network**(1./2.2)/2.)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## input envmaps

In [40]:
plt.figure(figsize=(15, 8))
ax = plt.subplot(221)
plt.imshow(im_uint8_halfenvmap)
ax.set_title('ori half env map')

ax = plt.subplot(222)
plt.imshow(im_envmap_ori_uint8)
ax.set_title('Ori env map (global RAW coords)')

ax = plt.subplot(224)
plt.imshow(recImg_gen_uint8)
ax.set_title('render SG env map (global RAW coords)')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …